In [2]:
#########START
#spark.stop()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
jan = spark.read.csv('../bigdata/yellow_tripdata_2017-01.csv', header=True, inferSchema = True)

loc = spark.read.csv('taxi _zone_lookup.csv', header=True, inferSchema = True)


In [3]:
import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|      Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-01-09|11:25:45|
|    186|Manhattan|Penn Station/Madi...|    234|Manhattan|            Union Sq|              1|     0.9|         1| 

In [4]:
byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(1))

Graph = df_joined.select('PUZone', 'DOZone', 'month')
Graph = Graph.groupBy('PUZone', 'DOZone').count()
Graph = Graph.withColumn("month",lit(1))
Graph = Graph.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Graph = Graph.join(byTotalFare,[Graph.src==byTotalFare.PUZone, Graph.dst==byTotalFare.DOZone],'left_outer')
Graph = Graph.drop('PUZone', 'DOZone')
Graph = Graph.withColumnRenamed("avg(total_fare)", "avg_fare")
Graph.show()

+--------------------+--------------------+-----+-----+------------------+
|                 src|                 dst|count|month|          avg_fare|
+--------------------+--------------------+-----+-----+------------------+
|Allerton/Pelham G...|        East Tremont|    1|    1|              11.8|
|        Battery Park|         Fort Greene|   10|    1|20.003000000000004|
|        Battery Park|     Windsor Terrace|    4|    1|            30.445|
|   Battery Park City|        East Village| 1716|    1| 18.92319930069934|
|        Bloomingdale|           Chinatown|    7|    1| 32.61714285714286|
|        Bloomingdale|     Mariners Harbor|    1|    1|             85.42|
|           Bronxdale|         Eastchester|    1|    1|               9.3|
|  Brooklyn Navy Yard|          Kensington|    1|    1|              18.3|
|      Bushwick North|    Hamilton Heights|    4|    1|           53.8475|
|      Bushwick South|Long Island City/...|    5|    1|            22.432|
|     Carroll Gardens|Upp

In [5]:

########## max distance
max_dist = df_joined.orderBy('distance', ascending=False)
max_dist = max_dist.limit(10)
max_dist = max_dist.withColumnRenamed("distance", "max_distance")
max_dist = max_dist.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist.show()

+--------------------+--------------------+------------+-----+
|              PUZone|              DOZone|max_distance|month|
+--------------------+--------------------+------------+-----+
|Springfield Garde...|                  NA|      264.71|    1|
|          Ocean Hill|Two Bridges/Sewar...|       180.7|    1|
|         JFK Airport| UN/Turtle Bay South|       173.2|    1|
|     Mariners Harbor|                  NA|      156.17|    1|
|        Clinton East|                  NA|       151.7|    1|
|Flushing Meadows-...|                  NA|      143.45|    1|
|                  NA|                  NA|       143.4|    1|
|         JFK Airport|                  NA|       143.4|    1|
|       Richmond Hill|                  NA|       141.8|    1|
|         JFK Airport|                  NA|      139.17|    1|
+--------------------+--------------------+------------+-----+



In [6]:

######max fare
max_fare = df_joined.orderBy('total_fare', ascending=False)
max_fare = max_fare.limit(10)
max_fare = max_fare.withColumnRenamed("total_fare", "max_fare")
max_fare = max_fare.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare.show()

+--------------------+--------------------+--------+-----+
|              PUZone|              DOZone|max_fare|month|
+--------------------+--------------------+--------+-----+
|Upper West Side S...|                  NV|625901.6|    1|
|Two Bridges/Sewar...|Washington Height...|538580.0|    1|
|Upper East Side S...|     Lenox Hill West|  9001.3|    1|
|                  NV|                  NV| 8043.84|    1|
|       Midtown North|Times Sq/Theatre ...|  3009.8|    1|
|                  NV|                  NV|  3009.3|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 2000.28|    1|
|           Sunnyside|           Sunnyside| 1000.29|    1|
+--------------------+--------------------+--------+-----+



In [7]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))
df_joined = df_joined.withColumn('PUClock', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))

df_joined.show(10)

+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|           DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|PUClock|
+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|   Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-01-09|11:25:45|    1| morning|     11|
|    186|Manhattan|Penn Station/Madi

In [8]:
###### by time
PUByTime = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTime = PUByTime.orderBy('count', ascending=False)
PUByTime.show(10)

+---------+--------------------+-----+------+
|   PUHour|              PUZone|month| count|
+---------+--------------------+-----+------+
|afternoon|Upper East Side S...|    1|157826|
|afternoon|Upper East Side N...|    1|151358|
|  evening|Times Sq/Theatre ...|    1|136867|
|  evening|      Midtown Center|    1|136487|
|afternoon|      Midtown Center|    1|134284|
|  evening|            Union Sq|    1|129806|
|  evening|        Midtown East|    1|123194|
|  morning|Upper East Side N...|    1|115309|
|afternoon|        Midtown East|    1|110824|
|  evening|        East Village|    1|110726|
+---------+--------------------+-----+------+
only showing top 10 rows



In [9]:
####### best PU locs
best_locs = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.orderBy('count', ascending=False)
best_locs.show(10)

+--------------------+-----+------+
|              PUZone|month| count|
+--------------------+-----+------+
|Upper East Side S...|    1|380663|
|Upper East Side N...|    1|363649|
|      Midtown Center|    1|360995|
|Penn Station/Madi...|    1|339970|
|Times Sq/Theatre ...|    1|334079|
|            Union Sq|    1|333035|
|        Midtown East|    1|330946|
|         Murray Hill|    1|306244|
|        East Village|    1|303580|
|        Clinton East|    1|297414|
+--------------------+-----+------+
only showing top 10 rows



In [10]:
vertexP = df_joined.select("PUZone").distinct()
vertexP = vertexP.selectExpr("PUZone as Zone")
print(vertexP.count())
vertexD = df_joined.select("DOZone").distinct()
vertexD = vertexD.selectExpr("DOZone as Zone")
print(vertexD.count())
if vertexD.count() > vertexP.count():
  vertex = vertexD.unionAll(vertexP)
else : vertex = vertexP.unionAll(vertexD)
print(vertex.count())
vertex = vertex.select("Zone").distinct()
print(vertex.count())
vertex.toPandas().to_csv('Zones.csv', index=False)
vertex.show()

258
261
519
261
+--------------------+
|                Zone|
+--------------------+
|           Homecrest|
|Governor's Island...|
|              Corona|
|    Bensonhurst West|
|         Westerleigh|
|      Newark Airport|
|          Douglaston|
|Charleston/Totten...|
|          Mount Hope|
|      Pelham Parkway|
|East Concourse/Co...|
|         Marble Hill|
|           Rego Park|
|Upper East Side S...|
|       Dyker Heights|
|Heartland Village...|
|   Kew Gardens Hills|
|     Jackson Heights|
|             Bayside|
|TriBeCa/Civic Center|
+--------------------+
only showing top 20 rows



In [11]:
edge = df_joined.groupBy('PUZone', 'DOZone').count()
edge = edge.selectExpr("PUZone as src", "DOZone as dst", "count as weight")
edge.toPandas().to_csv('Edges_jan.csv', index=False)
edge.show()

+--------------------+--------------------+------+
|                 src|                 dst|weight|
+--------------------+--------------------+------+
|        Midtown East|Meatpacking/West ...|  1918|
|           Sunnyside|         Old Astoria|   246|
|            Gramercy|            Flatiron|  7191|
|        East Village|         JFK Airport|  1107|
|     Jackson Heights|        Clinton East|    23|
|   East Harlem North|            Union Sq|   159|
| UN/Turtle Bay South|          Park Slope|   300|
|        East Village|              Corona|    56|
|Times Sq/Theatre ...|Long Island City/...|   905|
|       Midtown North|            Flatiron|  2685|
|TriBeCa/Civic Center|     Lenox Hill East|   983|
|            Union Sq|Financial Distric...|  2575|
|Sutton Place/Turt...|             Maspeth|    78|
|Upper West Side S...|    Garment District|  1562|
|         Old Astoria|Times Sq/Theatre ...|    51|
|   Battery Park City|        East Village|  1716|
|Central Harlem North|         

In [12]:
#####################################################  
### Feb

jan = spark.read.csv('../bigdata/yellow_tripdata_2017-02.csv', header=True, inferSchema = True)
jan = jan.drop('VendorID','store_and_fwd_flag' ,'improvement_surcharge')

import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')


In [13]:
df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|     87|Manhattan|Financial Distric...|    246|Manhattan|West Chelsea/Huds...|              1|    3.29|         1|   87|  246|           1|14.0|  0.0|    0.5| 1.5| 0.0|      16.3|2017-02-03|02:03:50|2017-02-03|02:20:55|
|     68|Manhattan|        East Chelsea|    239|Manhattan|Upper West Side S...|              1|     2.8|         1| 

In [14]:
byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(2))

Grapht = df_joined.select('PUZone', 'DOZone', 'month')
Grapht = Grapht.groupBy('PUZone', 'DOZone').count()
Grapht = Grapht.withColumn("month",lit(2))
Grapht = Grapht.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Grapht = Grapht.join(byTotalFare,[Grapht.src==byTotalFare.PUZone, Grapht.dst==byTotalFare.DOZone],'left_outer')
Grapht = Grapht.drop('PUZone', 'DOZone')
Grapht = Grapht.withColumnRenamed("avg(total_fare)", "avg_fare")
Graph = Graph.union(Grapht)
Graph.show()

+--------------------+--------------------+-----+-----+------------------+
|                 src|                 dst|count|month|          avg_fare|
+--------------------+--------------------+-----+-----+------------------+
|Allerton/Pelham G...|        East Tremont|    1|    1|              11.8|
|        Battery Park|         Fort Greene|   10|    1|20.003000000000004|
|        Battery Park|     Windsor Terrace|    4|    1|            30.445|
|   Battery Park City|        East Village| 1716|    1| 18.92319930069934|
|        Bloomingdale|           Chinatown|    7|    1| 32.61714285714286|
|        Bloomingdale|     Mariners Harbor|    1|    1|             85.42|
|           Bronxdale|         Eastchester|    1|    1|               9.3|
|  Brooklyn Navy Yard|          Kensington|    1|    1|              18.3|
|      Bushwick North|    Hamilton Heights|    4|    1|           53.8475|
|      Bushwick South|Long Island City/...|    5|    1|            22.432|
|     Carroll Gardens|Upp

In [15]:
########## max distance
max_distt = df_joined.orderBy('distance', ascending=False)
max_distt = max_distt.limit(10)
max_distt = max_distt.withColumnRenamed("distance", "max_distance")
max_distt = max_distt.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist = max_dist.union(max_distt)
max_dist.show()

+--------------------+--------------------+------------+-----+
|              PUZone|              DOZone|max_distance|month|
+--------------------+--------------------+------------+-----+
|Springfield Garde...|                  NA|      264.71|    1|
|          Ocean Hill|Two Bridges/Sewar...|       180.7|    1|
|         JFK Airport| UN/Turtle Bay South|       173.2|    1|
|     Mariners Harbor|                  NA|      156.17|    1|
|        Clinton East|                  NA|       151.7|    1|
|Flushing Meadows-...|                  NA|      143.45|    1|
|                  NA|                  NA|       143.4|    1|
|         JFK Airport|                  NA|       143.4|    1|
|       Richmond Hill|                  NA|       141.8|    1|
|         JFK Airport|                  NA|      139.17|    1|
|Prospect-Lefferts...|        Clinton Hill|       324.0|    2|
|           Bay Ridge|                  NA|      298.47|    2|
|             Bedford|             Bedford|       282.0

In [16]:
######max fare
max_faret = df_joined.orderBy('total_fare', ascending=False)
max_faret = max_faret.limit(10)
max_faret = max_faret.withColumnRenamed("total_fare", "max_fare")
max_faret = max_faret.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare = max_fare.union(max_faret)
max_fare.show()

+--------------------+--------------------+---------+-----+
|              PUZone|              DOZone| max_fare|month|
+--------------------+--------------------+---------+-----+
|Upper West Side S...|                  NV| 625901.6|    1|
|Two Bridges/Sewar...|Washington Height...| 538580.0|    1|
|Upper East Side S...|     Lenox Hill West|   9001.3|    1|
|                  NV|                  NV|  8043.84|    1|
|       Midtown North|Times Sq/Theatre ...|   3009.8|    1|
|                  NV|                  NV|   3009.3|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  1000.29|    1|
|            Union Sq|      Midtown Center|361772.02|    2|
|Upper East Side N...|Upper West Side S...|335545.21|    2|
|Upper West Side N...|      Midtown Center|171856.99|    2|
|Upper East Side N...|            Gramer

In [17]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))

df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|   PUHour|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+---------+
|     87|Manhattan|Financial Distric...|    246|Manhattan|West Chelsea/Huds...|              1|    3.29|         1|   87|  246|           1|14.0|  0.0|    0.5| 1.5| 0.0|      16.3|2017-02-03|02:03:50|2017-02-03|02:20:55|    2| midnight|
|     68|Manhattan|        East Chelsea|    239|Manh

In [18]:
###### by time
PUByTimet = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTimet = PUByTimet.orderBy('count', ascending=False)
PUByTime = PUByTime.union(PUByTimet)
PUByTime.show()

+---------+--------------------+-----+------+
|   PUHour|              PUZone|month| count|
+---------+--------------------+-----+------+
|afternoon|Upper East Side S...|    1|157826|
|afternoon|Upper East Side N...|    1|151358|
|  evening|Times Sq/Theatre ...|    1|136867|
|  evening|      Midtown Center|    1|136487|
|afternoon|      Midtown Center|    1|134284|
|  evening|            Union Sq|    1|129806|
|  evening|        Midtown East|    1|123194|
|  morning|Upper East Side N...|    1|115309|
|afternoon|        Midtown East|    1|110824|
|  evening|        East Village|    1|110726|
|afternoon|            Union Sq|    1|109299|
|  evening|Penn Station/Madi...|    1|108464|
|  evening|Upper East Side S...|    1|105797|
|  evening| Lincoln Square East|    1|104454|
|  morning|Penn Station/Madi...|    1|104143|
|  morning|Upper East Side S...|    1|103239|
|afternoon|Penn Station/Madi...|    1|102715|
|  evening|        Clinton East|    1|101075|
|afternoon|Upper West Side S...|  

In [19]:
####### best PU locs
best_locst = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.union(best_locst)
best_locs.show()

+--------------------+-----+------+
|              PUZone|month| count|
+--------------------+-----+------+
|Upper East Side S...|    1|380663|
|Upper East Side N...|    1|363649|
|      Midtown Center|    1|360995|
|Penn Station/Madi...|    1|339970|
|Times Sq/Theatre ...|    1|334079|
|            Union Sq|    1|333035|
|        Midtown East|    1|330946|
|         Murray Hill|    1|306244|
|        East Village|    1|303580|
|        Clinton East|    1|297414|
| Lincoln Square East|    1|277230|
|Upper West Side S...|    1|252264|
|       Midtown North|    1|250939|
|            Gramercy|    1|238695|
|       Midtown South|    1|227768|
|         JFK Airport|    1|227167|
|     Lenox Hill West|    1|226739|
|        East Chelsea|    1|224558|
|   LaGuardia Airport|    1|212193|
|        West Village|    1|211629|
+--------------------+-----+------+
only showing top 20 rows



In [20]:
###############################################
#### March

jan = spark.read.csv('../bigdata/yellow_tripdata_2017-03.csv', header=True, inferSchema = True)
jan = jan.drop('VendorID','store_and_fwd_flag' ,'improvement_surcharge')

import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')


In [21]:
df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+
|    148|Manhattan|     Lower East Side|     48|Manhattan|        Clinton East|              1|    4.06|         1|  148|   48|           1|14.0|  0.5|    0.5|3.06| 0.0|     18.36|2017-03-09|21:30:11|2017-03-09|21:44:20|
|     48|Manhattan|        Clinton East|    107|Manhattan|            Gramercy|              1|    2.73|         1| 

In [22]:
byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(3))

Grapht = df_joined.select('PUZone', 'DOZone', 'month')
Grapht = Grapht.groupBy('PUZone', 'DOZone').count()
Grapht = Grapht.withColumn("month",lit(3))
Grapht = Grapht.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Grapht = Grapht.join(byTotalFare,[Grapht.src==byTotalFare.PUZone, Grapht.dst==byTotalFare.DOZone],'left_outer')
Grapht = Grapht.drop('PUZone', 'DOZone')
Grapht = Grapht.withColumnRenamed("avg(total_fare)", "avg_fare")
Graph = Graph.union(Grapht)
Graph.show()

+--------------------+--------------------+-----+-----+------------------+
|                 src|                 dst|count|month|          avg_fare|
+--------------------+--------------------+-----+-----+------------------+
|Allerton/Pelham G...|        East Tremont|    1|    1|              11.8|
|        Battery Park|         Fort Greene|   10|    1|20.003000000000004|
|        Battery Park|     Windsor Terrace|    4|    1|            30.445|
|   Battery Park City|        East Village| 1716|    1| 18.92319930069934|
|        Bloomingdale|           Chinatown|    7|    1| 32.61714285714286|
|        Bloomingdale|     Mariners Harbor|    1|    1|             85.42|
|           Bronxdale|         Eastchester|    1|    1|               9.3|
|  Brooklyn Navy Yard|          Kensington|    1|    1|              18.3|
|      Bushwick North|    Hamilton Heights|    4|    1|           53.8475|
|      Bushwick South|Long Island City/...|    5|    1|            22.432|
|     Carroll Gardens|Upp

In [23]:
########## max distance
max_distt = df_joined.orderBy('distance', ascending=False)
max_distt = max_distt.limit(10)
max_distt = max_distt.withColumnRenamed("distance", "max_distance")
max_distt = max_distt.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist = max_dist.union(max_distt)
max_dist.show()

+--------------------+--------------------+------------+-----+
|              PUZone|              DOZone|max_distance|month|
+--------------------+--------------------+------------+-----+
|Springfield Garde...|                  NA|      264.71|    1|
|          Ocean Hill|Two Bridges/Sewar...|       180.7|    1|
|         JFK Airport| UN/Turtle Bay South|       173.2|    1|
|     Mariners Harbor|                  NA|      156.17|    1|
|        Clinton East|                  NA|       151.7|    1|
|Flushing Meadows-...|                  NA|      143.45|    1|
|                  NA|                  NA|       143.4|    1|
|         JFK Airport|                  NA|       143.4|    1|
|       Richmond Hill|                  NA|       141.8|    1|
|         JFK Airport|                  NA|      139.17|    1|
|Prospect-Lefferts...|        Clinton Hill|       324.0|    2|
|           Bay Ridge|                  NA|      298.47|    2|
|             Bedford|             Bedford|       282.0

In [24]:
######max fare
max_faret = df_joined.orderBy('total_fare', ascending=False)
max_faret = max_faret.limit(10)
max_faret = max_faret.withColumnRenamed("total_fare", "max_fare")
max_faret = max_faret.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare = max_fare.union(max_faret)
max_fare.show()

+--------------------+--------------------+---------+-----+
|              PUZone|              DOZone| max_fare|month|
+--------------------+--------------------+---------+-----+
|Upper West Side S...|                  NV| 625901.6|    1|
|Two Bridges/Sewar...|Washington Height...| 538580.0|    1|
|Upper East Side S...|     Lenox Hill West|   9001.3|    1|
|                  NV|                  NV|  8043.84|    1|
|       Midtown North|Times Sq/Theatre ...|   3009.8|    1|
|                  NV|                  NV|   3009.3|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  1000.29|    1|
|            Union Sq|      Midtown Center|361772.02|    2|
|Upper East Side N...|Upper West Side S...|335545.21|    2|
|Upper West Side N...|      Midtown Center|171856.99|    2|
|Upper East Side N...|            Gramer

In [25]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+
|    148|Manhattan|     Lower East Side|     48|Manhattan|        Clinton East|              1|    4.06|         1|  148|   48|           1|14.0|  0.5|    0.5|3.06| 0.0|     18.36|2017-03-09|21:30:11|2017-03-09|21:44:20|    3| evening|
|     48|Manhattan|        Clinton East|    107|Manhatta

In [26]:
###### by time
PUByTimet = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTimet = PUByTimet.orderBy('count', ascending=False)
PUByTime = PUByTime.union(PUByTimet)
PUByTime.show()

+---------+--------------------+-----+------+
|   PUHour|              PUZone|month| count|
+---------+--------------------+-----+------+
|afternoon|Upper East Side S...|    1|157826|
|afternoon|Upper East Side N...|    1|151358|
|  evening|Times Sq/Theatre ...|    1|136867|
|  evening|      Midtown Center|    1|136487|
|afternoon|      Midtown Center|    1|134284|
|  evening|            Union Sq|    1|129806|
|  evening|        Midtown East|    1|123194|
|  morning|Upper East Side N...|    1|115309|
|afternoon|        Midtown East|    1|110824|
|  evening|        East Village|    1|110726|
|afternoon|            Union Sq|    1|109299|
|  evening|Penn Station/Madi...|    1|108464|
|  evening|Upper East Side S...|    1|105797|
|  evening| Lincoln Square East|    1|104454|
|  morning|Penn Station/Madi...|    1|104143|
|  morning|Upper East Side S...|    1|103239|
|afternoon|Penn Station/Madi...|    1|102715|
|  evening|        Clinton East|    1|101075|
|afternoon|Upper West Side S...|  

In [27]:
####### best PU locs
best_locst = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.union(best_locst)
best_locs.show()

+--------------------+-----+------+
|              PUZone|month| count|
+--------------------+-----+------+
|Upper East Side S...|    1|380663|
|Upper East Side N...|    1|363649|
|      Midtown Center|    1|360995|
|Penn Station/Madi...|    1|339970|
|Times Sq/Theatre ...|    1|334079|
|            Union Sq|    1|333035|
|        Midtown East|    1|330946|
|         Murray Hill|    1|306244|
|        East Village|    1|303580|
|        Clinton East|    1|297414|
| Lincoln Square East|    1|277230|
|Upper West Side S...|    1|252264|
|       Midtown North|    1|250939|
|            Gramercy|    1|238695|
|       Midtown South|    1|227768|
|         JFK Airport|    1|227167|
|     Lenox Hill West|    1|226739|
|        East Chelsea|    1|224558|
|   LaGuardia Airport|    1|212193|
|        West Village|    1|211629|
+--------------------+-----+------+
only showing top 20 rows



In [28]:
#################################################################
##### April
jan = spark.read.csv('../bigdata/yellow_tripdata_2017-04.csv', header=True, inferSchema = True)
jan = jan.drop('VendorID','store_and_fwd_flag' ,'improvement_surcharge')

import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

In [29]:
df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+-----+----+----------+----------+--------+----------+--------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax|  tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+-----+----+----------+----------+--------+----------+--------+
|    158|Manhattan|Meatpacking/West ...|    113|Manhattan|Greenwich Village...|              1|     1.8|         1|  158|  113|           2|10.5|  0.5|    0.5|  0.0| 0.0|      11.8|2017-04-01|00:00:00|2017-04-01|00:15:07|
|     87|Manhattan|Financial Distric...|    158|Manhattan|Meatpacking/West ...|              3|     3.7|        

In [30]:
byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
byDistance = df_joined.groupBy('PUZone', 'DOZone').max('distance')

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(4))

Grapht = df_joined.select('PUZone', 'DOZone', 'month')
Grapht = Grapht.groupBy('PUZone', 'DOZone').count()
Grapht = Grapht.withColumn("month",lit(4))
Grapht = Grapht.selectExpr("PUZone as src", "DOZone as dst", "count as count", "month as month")

byTotalFare = df_joined.groupBy('PUZone', 'DOZone').avg('total_fare')
Grapht = Grapht.join(byTotalFare,[Grapht.src==byTotalFare.PUZone, Grapht.dst==byTotalFare.DOZone],'left_outer')
Grapht = Grapht.drop('PUZone', 'DOZone')
Grapht = Grapht.withColumnRenamed("avg(total_fare)", "avg_fare")
Graph = Graph.union(Grapht)
Graph.show()

+--------------------+--------------------+-----+-----+------------------+
|                 src|                 dst|count|month|          avg_fare|
+--------------------+--------------------+-----+-----+------------------+
|Allerton/Pelham G...|        East Tremont|    1|    1|              11.8|
|        Battery Park|         Fort Greene|   10|    1|20.003000000000004|
|        Battery Park|     Windsor Terrace|    4|    1|            30.445|
|   Battery Park City|        East Village| 1716|    1| 18.92319930069934|
|        Bloomingdale|           Chinatown|    7|    1| 32.61714285714286|
|        Bloomingdale|     Mariners Harbor|    1|    1|             85.42|
|           Bronxdale|         Eastchester|    1|    1|               9.3|
|  Brooklyn Navy Yard|          Kensington|    1|    1|              18.3|
|      Bushwick North|    Hamilton Heights|    4|    1|           53.8475|
|      Bushwick South|Long Island City/...|    5|    1|            22.432|
|     Carroll Gardens|Upp

In [31]:
########## max distance
max_distt = df_joined.orderBy('distance', ascending=False)
max_distt = max_distt.limit(10)
max_distt = max_distt.withColumnRenamed("distance", "max_distance")
max_distt = max_distt.select('PUZone', 'DOZone', 'max_distance', 'month')
max_dist = max_dist.union(max_distt)
max_dist.show()

+--------------------+--------------------+------------+-----+
|              PUZone|              DOZone|max_distance|month|
+--------------------+--------------------+------------+-----+
|Springfield Garde...|                  NA|      264.71|    1|
|          Ocean Hill|Two Bridges/Sewar...|       180.7|    1|
|         JFK Airport| UN/Turtle Bay South|       173.2|    1|
|     Mariners Harbor|                  NA|      156.17|    1|
|        Clinton East|                  NA|       151.7|    1|
|Flushing Meadows-...|                  NA|      143.45|    1|
|                  NA|                  NA|       143.4|    1|
|         JFK Airport|                  NA|       143.4|    1|
|       Richmond Hill|                  NA|       141.8|    1|
|         JFK Airport|                  NA|      139.17|    1|
|Prospect-Lefferts...|        Clinton Hill|       324.0|    2|
|           Bay Ridge|                  NA|      298.47|    2|
|             Bedford|             Bedford|       282.0

In [32]:
######max fare
max_faret = df_joined.orderBy('total_fare', ascending=False)
max_faret = max_faret.limit(10)
max_faret = max_faret.withColumnRenamed("total_fare", "max_fare")
max_faret = max_faret.select('PUZone', 'DOZone', 'max_fare', 'month')
max_fare = max_fare.union(max_faret)
max_fare.show()

+--------------------+--------------------+---------+-----+
|              PUZone|              DOZone| max_fare|month|
+--------------------+--------------------+---------+-----+
|Upper West Side S...|                  NV| 625901.6|    1|
|Two Bridges/Sewar...|Washington Height...| 538580.0|    1|
|Upper East Side S...|     Lenox Hill West|   9001.3|    1|
|                  NV|                  NV|  8043.84|    1|
|       Midtown North|Times Sq/Theatre ...|   3009.8|    1|
|                  NV|                  NV|   3009.3|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  2000.28|    1|
|           Sunnyside|           Sunnyside|  1000.29|    1|
|            Union Sq|      Midtown Center|361772.02|    2|
|Upper East Side N...|Upper West Side S...|335545.21|    2|
|Upper West Side N...|      Midtown Center|171856.99|    2|
|Upper East Side N...|            Gramer

In [33]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))
df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+-----+----+----------+----------+--------+----------+--------+-----+---------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax|  tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|   PUHour|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+-----+----+----------+----------+--------+----------+--------+-----+---------+
|    158|Manhattan|Meatpacking/West ...|    113|Manhattan|Greenwich Village...|              1|     1.8|         1|  158|  113|           2|10.5|  0.5|    0.5|  0.0| 0.0|      11.8|2017-04-01|00:00:00|2017-04-01|00:15:07|    4| midnight|
|     87|Manhattan|Financial Distric...|    158|

In [34]:
###### by time
PUByTimet = df_joined.groupBy('PUHour', 'PUZone', 'month').count()
PUByTimet = PUByTimet.orderBy('count', ascending=False)
PUByTime = PUByTime.union(PUByTimet)
PUByTime.show()

+---------+--------------------+-----+------+
|   PUHour|              PUZone|month| count|
+---------+--------------------+-----+------+
|afternoon|Upper East Side S...|    1|157826|
|afternoon|Upper East Side N...|    1|151358|
|  evening|Times Sq/Theatre ...|    1|136867|
|  evening|      Midtown Center|    1|136487|
|afternoon|      Midtown Center|    1|134284|
|  evening|            Union Sq|    1|129806|
|  evening|        Midtown East|    1|123194|
|  morning|Upper East Side N...|    1|115309|
|afternoon|        Midtown East|    1|110824|
|  evening|        East Village|    1|110726|
|afternoon|            Union Sq|    1|109299|
|  evening|Penn Station/Madi...|    1|108464|
|  evening|Upper East Side S...|    1|105797|
|  evening| Lincoln Square East|    1|104454|
|  morning|Penn Station/Madi...|    1|104143|
|  morning|Upper East Side S...|    1|103239|
|afternoon|Penn Station/Madi...|    1|102715|
|  evening|        Clinton East|    1|101075|
|afternoon|Upper West Side S...|  

In [35]:
####### best PU locs
best_locst = df_joined.groupBy('PUZone', 'month').count()
best_locs = best_locs.union(best_locst)
best_locs.show()

+--------------------+-----+------+
|              PUZone|month| count|
+--------------------+-----+------+
|Upper East Side S...|    1|380663|
|Upper East Side N...|    1|363649|
|      Midtown Center|    1|360995|
|Penn Station/Madi...|    1|339970|
|Times Sq/Theatre ...|    1|334079|
|            Union Sq|    1|333035|
|        Midtown East|    1|330946|
|         Murray Hill|    1|306244|
|        East Village|    1|303580|
|        Clinton East|    1|297414|
| Lincoln Square East|    1|277230|
|Upper West Side S...|    1|252264|
|       Midtown North|    1|250939|
|            Gramercy|    1|238695|
|       Midtown South|    1|227768|
|         JFK Airport|    1|227167|
|     Lenox Hill West|    1|226739|
|        East Chelsea|    1|224558|
|   LaGuardia Airport|    1|212193|
|        West Village|    1|211629|
+--------------------+-----+------+
only showing top 20 rows



In [36]:
import pandas as pd
Graph.toPandas().to_csv('Graph.csv', index=False)

In [37]:
max_dist.toPandas().to_csv('Max_distance.csv', index=False)

In [38]:
max_fare.toPandas().to_csv('Max_fare.csv', index=False)

In [39]:
PUByTime.toPandas().to_csv('PickUpByTime.csv', index=False)

In [40]:
best_locs.toPandas().to_csv('Best_PickUp_Locs.csv', index=False)

In [41]:
spark.stop()

In [42]:
#######################Cluster

import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
jan = spark.read.csv('../bigdata/yellow_tripdata_2017-01.csv', header=True, inferSchema = True)

loc = spark.read.csv('taxi _zone_lookup.csv', header=True, inferSchema = True)

In [43]:
import pyspark.sql.functions

split_col = pyspark.sql.functions.split(jan['tpep_pickup_datetime'], ' ')
jan = jan.withColumn('PUDate', split_col.getItem(0))
jan = jan.withColumn('PUTime', split_col.getItem(1))
split_col = pyspark.sql.functions.split(jan['tpep_dropoff_datetime'], ' ')
jan = jan.withColumn('DODate', split_col.getItem(0))
jan = jan.withColumn('DOTime', split_col.getItem(1))

jan = jan.drop('tpep_pickup_datetime','tpep_dropoff_datetime')

df_joined = jan.join(loc,[jan.PULocationID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("LocationID as PULocID", "Borough as PUBorough", "Zone as PUZone", "passenger_count as passenger_count", "trip_distance as distance", "RateCodeID as RateCodeID", "PULocationID as srcID", "DOLocationID as dstID", "payment_type as payment_type", "fare_amount as fare", "extra as extra", "mta_tax as mta_tax", "tip_amount as tip", "tolls_amount as toll", "total_amount as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")
df_joined = df_joined.join(loc,[df_joined.dstID==loc.LocationID],'left_outer')
df_joined = df_joined.selectExpr("PULocID as PULocID", "PUBorough as PUBorough", "PUZone as PUZone", "LocationID as DOLocID", "Borough as DOBorough", "Zone as DOZone", "passenger_count as passenger_count", "distance as distance", "RateCodeID as RateCodeID", "srcID as srcID", "dstID as dstID", "payment_type as payment_type", "fare as fare", "extra as extra", "mta_tax as mta_tax", "tip as tip", "toll as toll", "total_fare as total_fare", "PUDate as PUDate", "PUTime as PUTime", "DODate as DODate", "DOTime as DOTime")

from pyspark.sql.functions import lit
df_joined = df_joined.withColumn("month",lit(1))

df_joined.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+
|    263|Manhattan|      Yorkville West|    161|Manhattan|      Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-01-09|11:25:45|    1|
|    186|Manhattan|Penn Station/Madi...|    234|Manhattan|            Union Sq|             

In [44]:
#### manipulate time
split_col = pyspark.sql.functions.split(df_joined['PUTime'], ':')
df_joined = df_joined.withColumn('PUHour', split_col.getItem(0))
df_joined = df_joined.withColumn('PUClock', split_col.getItem(0))

from pyspark.sql.functions import when

df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=0) & (df_joined["PUHour"] <6), 'midnight').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=6) & (df_joined["PUHour"] <12), 'morning').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=12) & (df_joined["PUHour"] <18), 'afternoon').otherwise(df_joined["PUHour"]))
df_joined = df_joined.withColumn("PUHour", when((df_joined["PUHour"] >=18) & (df_joined["PUHour"] <=23), 'evening').otherwise(df_joined["PUHour"]))

df_joined.show(10)

+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|           DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|PUClock|
+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|   Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-01-09|11:25:45|    1| morning|     11|
|    186|Manhattan|Penn Station/Madi

In [45]:
from pyspark.sql.types import IntegerType
df_joined = df_joined.withColumn("PUClock", df_joined["PUClock"].cast(IntegerType()))
df_joined.show(10)

+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|           DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|PUClock|
+-------+---------+--------------------+-------+---------+-----------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|   Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-01-09|11:25:45|    1| morning|     11|
|    186|Manhattan|Penn Station/Madi

In [46]:
from pyspark.ml.feature import VectorAssembler
#vecAssembler = VectorAssembler(inputCols=["PULocID", "PUBorough", "PUZone", "DOLocID","DOBorough", "DOZone", "passenger_count" ,"distance" , "RateCodeID", "srcID", "dstID", "payment_type", "fare" ,"extra", "mta_tax",  "tip","toll","total_fare", "PUDate", "PUTime","DODate","DOTime","month" ,"PUHour"], outputCol="features")
vecAssembler = VectorAssembler(inputCols=["passenger_count" ,"distance" , "RateCodeID", "srcID", "dstID", "payment_type", "fare" ,"extra", "mta_tax","tip","toll","total_fare","month", "PUClock"], outputCol="features")
new_df = vecAssembler.transform(df_joined)
new_df.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+--------------------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|PUClock|            features|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+--------------------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|      Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|      15.3|2017-01-09|11:13:28|2017-

In [47]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=3, seed=1)  # 2 clusters here
model = kmeans.fit(new_df.select('features'))

In [48]:
transformed = model.transform(new_df)
transformed.show()

+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+--------------------+----------+
|PULocID|PUBorough|              PUZone|DOLocID|DOBorough|              DOZone|passenger_count|distance|RateCodeID|srcID|dstID|payment_type|fare|extra|mta_tax| tip|toll|total_fare|    PUDate|  PUTime|    DODate|  DOTime|month|  PUHour|PUClock|            features|prediction|
+-------+---------+--------------------+-------+---------+--------------------+---------------+--------+----------+-----+-----+------------+----+-----+-------+----+----+----------+----------+--------+----------+--------+-----+--------+-------+--------------------+----------+
|    263|Manhattan|      Yorkville West|    161|Manhattan|      Midtown Center|              1|     3.3|         1|  263|  161|           1|12.5|  0.0|    0.5| 2.0| 0.0|   

In [49]:
# load function
from pyspark.sql import functions as F
import pandas as pd

# aggregate data
stat = transformed.groupby('prediction').agg(
    F.avg(F.col('total_fare')).alias('avg_fare'),
    F.avg(F.col('distance')).alias('avg_distance'),
    F.avg(F.col('passenger_count')).alias('avg_passenger'),
    F.avg(F.col('tip')).alias('avg_tip'),
    F.variance(F.col('total_fare')).alias('var_fare'),
    F.variance(F.col('distance')).alias('var_distance'),
    F.variance(F.col('passenger_count')).alias('var_passenger'),
    F.variance(F.col('tip')).alias('var_tip')
)
stat.toPandas().to_csv('Cluster_stat_jan.csv', index=False)
stat.show()

+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+
|prediction|          avg_fare|      avg_distance|     avg_passenger|          avg_tip|           var_fare|      var_distance|     var_passenger|          var_tip|
+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+
|         1|          582240.8|               0.0|               2.0|              0.0|3.812530913279998E9|               0.0|               2.0|              0.0|
|         2|14.176325674156118|2.4422242609346267|1.6243608399910456|1.611476741317611| 153.42641663155626|10.230785947969945|1.6082686820964247|6.093270927128124|
|         0|16.914793870864997| 3.269397492359212|1.6346444529247215|1.922589285284459|  243.2696916855305|16.115425474835977|1.6297981608965104|7.192955951568473|
+----------+----

In [50]:
spark.stop()